### 1. Import Libraries

In [1]:
import os
import sys
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from numba import cuda
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

Matplotlib is building the font cache; this may take a moment.


### 2. Import from mlcblab

In [2]:
from mlcvlab.models.nn4 import NN4
from mlcvlab.nn.losses import l2
from mlcvlab.optim.sgd import SGD
from mlcvlab.optim.sync_sgd import sync_sgd
# TODO: Import all the necessary code from mlcvlab package as you need... 

### 3. Set Seed

In [3]:
np.random.seed(42)

### 4. Helper functions

In [4]:
def load_dataset():
    x, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
    y = np.asarray(y, dtype=np.float64)
    return x, y

def prepare_data(x, y):
    y = np.apply_along_axis(lambda num: num % 2 == 0, 0, y).astype("int")
    return x, y

def split_train_test(x,y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=10000, random_state=0)
    return X_train, X_test, y_train, y_test

def minibatch(X_train,y_train,K):

    X_train_batches = []
    y_train_batches = []
    y_train=y_train.reshape(-1,1)
    
    assert len(X_train) == len(y_train), "Input and output data should have the same length."
    data_size = len(X_train)
    indices = np.arange(data_size)
    np.random.shuffle(indices)#random shuffling for distributed data

    for start_idx in range(0, data_size, K):
        end_idx = min(start_idx + K, data_size)
        batch_indices = indices[start_idx:end_idx]
        mini_batch_x=X_train[batch_indices,:]
        mini_batch_y=y_train[batch_indices,:]
        X_train_batches.append(mini_batch_x)
        y_train_batches.append(mini_batch_y)

    return X_train_batches, y_train_batches

def initialize_model():
    #TODO (Can use the similar approach used in HW1)
    # e.g. He Initialization for W0-W2, Xavier Initialization for W3
    # Also, initialize your model with a dropout parameter of 0.25 and use_batchnorm being true.
    
    #Xavier Weight Initialization
    n_in_1= 300
    n_out_1=784
    std_1 = np.sqrt(2) / np.sqrt(n_in_1 + n_out_1)
    W1 = np.random.normal(loc=0, scale=std_1, size=(n_in_1, n_out_1)) #(300,784)
    
    n_2 = 150
    std_2 = np.sqrt(2) / np.sqrt(n_2 + n_in_1)
    W2 = np.random.normal(loc=0, scale=std_2, size=(n_2, n_in_1)) #(150,300)
    
    n_3 = 50
    std_3 = np.sqrt(2) / np.sqrt(n_3 + n_2)
    W3 = np.random.normal(loc=0, scale=std_1, size=(n_3, n_2)) #(50,150)
    
    n_4 = 1
    std_4 = np.sqrt(2) / np.sqrt(n_4 + n_3)
    W4 = np.random.normal(loc=0, scale=std_1, size=(n_4, n_3)) #(1,50)
    
    print(f"Size of W1 : {W1.shape}, Size of W2 : {W2.shape}, Size of W3 : {W3.shape}, Size of W4 : {W4.shape}")
    four_layer_nn  = NN4(use_batchnorm=True, dropout_param=0.25)
    four_layer_nn.layers[0].W = W1
    four_layer_nn.layers[1].W = W2
    four_layer_nn.layers[2].W = W3
    four_layer_nn.layers[3].W = W4

    return four_layer_nn

def train_model(model, X_train_batches, y_train_batches):
    print("The gradient of emperical risk for W1, W2, W3 and W4:")
    griddim = (64,64)
    blockdim = (256,256)

    stream = cuda.stream()

    d_X_train_batches = cuda.to_device(X_train_batches, stream=stream)
    d_y_train_batches = cuda.to_device(y_train_batches, stream=stream)

    model_sync = sync_sdg[griddim, blockdim](model, d_X_train_batches, d_y_train_batches)
    model_sync = model_sync.copy_to_host()

    model_async = async_sgd[griddim, blockdim, stream](model, d_X_train_batches, d_y_train_batches)
    model_async = model_async.copy_to_host(stream=stream)

    return model_async, model_sync

def test_model(model, X_test, y_test):
    
    y_hat = model.nn4(X_test, mode='test')[0].T
    y_test = y_test.reshape(-1, 1)
    error = np.round_(np.abs(y_test - y_hat), 2)
    indicator_fn = np.greater(error, np.zeros(error.shape)).astype('float')

    accuracy = np.mean(indicator_fn) * 100
    return accuracy

### 5. Run the program

In [5]:
# minibatch size
K = 100

#load data
x, y = load_dataset()
print("Dataset imported successfully")

#prepare data
x, y = prepare_data(x,y)
print("Data prepared successfully")

# split data set
X_train, X_test, y_train, y_test = split_train_test(x,y)
print("Dataset Splitted successfully")

#initialize model
model = initialize_model()
print("NN4 successfully implemented")

#dividing data into mini batches
X_train_batches, y_train_batches = minibatch(X_train, y_train, K)
print("Data successfully split into mini batches")

#training model
sgd = SGD(model, X_train_batches, y_train_batches, lr=0.01, R=50)
print(f"Completed training with SGD, now testing...")

#testing model
accuracy = test_model(sgd, X_test, y_test)
print(f"Completed testing model using SGD - Accuracy : {accuracy}")

#training model with cuda
async_model, sync_model = train_model(model, X_train_batches, y_train_batches)
print(f"Completed training model - final W : {final_W}")
print(f"Completed training, now testing...")   

#testing model
# accuracy_async = test_model(model_async, X_test, y_test)
# print(f"Completed testing model using asynchronous SGD - Accuracy : {accuracy_async}")   

accuracy_sync = test_model(sync_model, X_test, y_test)
print(f"Completed testing model using synchronous SGD - Accuracy : {accuracy_sync}")

Dataset imported successfully
Data prepared successfully
Dataset Splitted successfully
Size of W1 : (300, 784), Size of W2 : (150, 300), Size of W3 : (50, 150), Size of W4 : (1, 50)
NN4 successfully implemented
Data successfully split into mini batches
[-0.10462909822566793, -16.122881208844447, -83.86079450929985, -1420.665243295562]
[0.0, -5.966810827177728, -139.75328859079667, -1345.7406939624082]
[0.0, -3.435045612197033, -160.5268225383835, -1835.7676665787983]
[0.0, -1.6523045805986214, -123.40516666124235, -1426.73408879528]
[0.0, -1.1872127661847316, -109.18658132334888, -1866.5512536828082]
[0.0, -0.8372060820413298, -89.9012116798002, -1906.4901496458344]
[0.0, -1.0012285145376232, -146.1525185277033, -1765.0932987616359]
[0.0, -0.72823443451925, -122.22810461468868, -1379.7208994781713]
[0.0, -0.6058883516918899, -132.94188179586897, -1506.7063520064473]
[0.0, -0.4830067559619795, -102.44113110335968, -1844.3668869235387]
[0.0, -0.4002495391142476, -75.4380568183295, -1319.

[0.0, -0.06312270969728623, -75.87747438331772, -1822.2038874275659]
[0.0, -0.047206130244017555, -57.01843627501704, -1856.1125561740628]
[0.0, -0.05652430416780252, -75.78155590618123, -1129.451569990803]
[0.0, -0.07403947929107556, -99.05878859624539, -1767.7196833698385]
[0.0, -0.051789148964692305, -63.31388824267967, -1328.5296484733458]
[0.0, -0.05931020355236396, -96.88826289868435, -2020.039684709483]
[0.0, -0.05993175149773594, -80.79750078087774, -1910.1761527115336]
[0.0, -0.047720787920652846, -76.55720283382765, -1209.573142405262]
[0.0, -0.08697801584406938, -92.91176817673758, -2125.173838791163]
[0.0, -0.06295212737762922, -82.08067154530475, -1205.553452735238]
[0.0, -0.04704028563904911, -65.87168937099403, -943.2539598415063]
[0.0, -0.07036699012748479, -92.89209972849739, -2852.1415626742846]
[0.0, -0.0434968247856469, -61.93697753026981, -1172.5339622634585]
[0.0, -0.05436606213764996, -85.40878376577956, -1651.7438368560681]
[0.0, -0.0518685309560572, -82.5494206

[0.0, -0.03357353532466095, -80.69155607336732, -1604.98597795146]
[0.0, -0.040456673804906945, -67.95564412161285, -1266.0235604396466]
[0.0, -0.04843313856835394, -122.04481926733486, -2113.142001882692]
[0.0, -0.026489992285801445, -58.46985875220421, -1446.670115125743]
[0.0, -0.02815061033567912, -62.273172174543824, -1767.2388379623535]
[0.0, -0.029821077008549666, -74.5762799615317, -1732.5738654433183]
[0.0, -0.037542060364732774, -79.86684169186613, -1686.715778863335]
[0.0, -0.02719470711795519, -67.37391609258255, -1403.6172661860148]
[0.0, -0.026216955566912982, -63.22159430917791, -1381.2742602520534]
[0.0, -0.0245646826043982, -55.07067465528903, -1346.1665061679546]
[0.0, -0.025691834259945593, -52.0607354281619, -1420.2471785892665]
[0.0, -0.0346476948951703, -74.74939631234932, -1403.185929725415]
[0.0, -0.028931281723093675, -72.59712213724751, -1696.3673248562898]
[0.0, -0.033696863376413956, -81.81142684347952, -2289.5711182008195]
[0.0, -0.028122960123938716, -59.2

[0.0, -0.02346649902153362, -81.98294786703893, -2117.6142089345294]
[0.0, -0.02121022353544147, -70.50944503447026, -1600.3486691652706]
[0.0, -0.022076550368384046, -83.05534366688725, -1804.4850143889937]
[0.0, -0.02559280914354895, -54.22187463267612, -1583.0053546361376]
[0.0, -0.024941456571409237, -69.682145061217, -1466.8756115977396]
[0.0, -0.02341957718758923, -83.11828417302769, -1543.506909568499]
[0.0, -0.021191435932048325, -62.9448602648332, -1377.0537302697596]
[0.0, -0.02258831043507993, -71.78408492078961, -2024.8295811589653]
[0.0, -0.021892582235203028, -63.386851369626584, -1572.1151037323646]
[0.0, -0.021006102757558626, -55.582571203167156, -1730.5823137085213]
[0.0, -0.03597705157874942, -89.20811324029593, -1932.2418055145836]
[0.0, -0.018841292798967714, -70.65094092832014, -1764.7235800317985]
[0.0, -0.01593181858277475, -47.73260027078091, -1725.934867449686]
[0.0, -0.022398957757740236, -81.08291443263914, -1653.8789473349252]
[0.0, -0.026324815471627187, -

[0.0, -0.015533877673455258, -43.445302250820575, -950.6625889490523]
[0.0, -0.01734913477541492, -65.33955395352996, -1732.0828966951863]
[0.0, -0.017872358347452457, -73.51814027466831, -1594.7010041979863]
[0.0, -0.03262789353001928, -92.40633658570405, -1970.928868796169]
[0.0, -0.02311322570806332, -79.0684084631176, -2057.8740338729685]
[0.0, -0.01856794014112976, -62.32167767659474, -1440.3586542870166]
[0.0, -0.014981251795771856, -50.03729830894963, -1401.2291161557666]
[0.0, -0.015430402573525076, -53.99783215334096, -1608.5308283658107]
[0.0, -0.021190996756676034, -83.78707238497844, -1707.2945050921269]
[0.0, -0.023118044384185747, -91.72911285927884, -1462.327797777314]
[0.0, -0.01452917148621675, -45.532000855761886, -1379.9283065166062]
[0.0, -0.016671195788371052, -75.88792335939598, -1789.1457868442321]
[0.0, -0.016393477124919703, -70.67072768685344, -1801.233156237324]
[0.0, -0.01561457502524579, -57.64531726037294, -1426.83101326245]
[0.0, -0.014191267845621263, -5

...

[0.0, -0.0004796028759455005, -27.842488797736728, -1603.2138914726715]
[0.0, -0.0005998967524357616, -38.99444131480428, -1838.6283340278917]
[0.0, -0.0009910685627683735, -63.92318009201812, -1687.8440264328756]
[0.0, -0.0005063503647578726, -27.11686112538726, -1625.6545351464988]
[0.0, -0.0005975520046832798, -34.28768612903661, -1588.0155992749324]
[0.0, -0.0005236316883185492, -35.85720777994791, -1636.374038019067]
[0.0, -0.0004708033792653947, -26.790487374455243, -1447.2239919053516]
[0.0, -0.0007354196551448064, -46.016314429721824, -1372.7865891520894]
[0.0, -0.0006381187298010951, -35.0016039046877, -1417.5872217570295]
[0.0, -0.00041224468551464053, -30.740801438737257, -1884.7618970450449]
[0.0, -0.0004391222339005567, -29.226809134711868, -1824.8440159072072]
[0.0, -0.0005650032588723763, -25.68154209009833, -1368.2092858741794]
[0.0, -0.0007781368540363425, -36.69091596587182, -1165.7210814130763]
[0.0, -0.0006177229776140706, -38.088518732131234, -1929.289767254499]
[0

[0.0, -0.0005802519002623322, -45.57193211650244, -1931.2139398701058]
[0.0, -0.00043708014282779637, -27.025552016788087, -1688.7710826453454]
[0.0, -0.0004934477674981688, -33.85648103236923, -1531.5581641193917]
[0.0, -0.0005853965526172605, -34.115914713157764, -1441.1419095092317]
[0.0, -0.00058361400983897, -39.63510542070764, -1578.8072611878142]
[0.0, -0.0004434227298535051, -30.573655325744415, -1671.1833256919763]
[0.0, -0.00046797824495698923, -23.32811454206181, -1183.3957110796766]
[0.0, -0.000622020443768146, -42.7950364752312, -1801.408822292929]
[0.0, -0.0004538184458781067, -35.8853484398577, -2025.5875324707783]
[0.0, -0.0004882607951636289, -42.189536435156995, -1853.7254982891784]
[0.0, -0.0005903643326418346, -41.899040115179396, -2092.128082824408]
[0.0, -0.0004663402431580704, -34.90505387631872, -1728.2854728140637]
[0.0, -0.0006199506010725176, -36.537052178590564, -1195.2992135048596]
[0.0, -0.0005542428599269689, -38.28535591041549, -1493.5778226163782]
[0.0,

[0.0, -0.0005753756764524178, -41.29031049379759, -1534.5432490332046]
[0.0, -0.0006219669793910343, -31.54394445214893, -1387.1678966422107]
[0.0, -0.0005721625578706392, -32.60865056608255, -1460.7505155719866]
[0.0, -0.0004847215607356702, -31.632231856712487, -1621.4558395372771]
[0.0, -0.0004522745279276531, -30.581735705068933, -1672.3459463344414]
[0.0, -0.0005012704308075443, -32.47768248293487, -1891.9208770600349]
[0.0, -0.0005422725686888385, -34.42398043722703, -1474.496337493523]
[0.0, -0.0005073727157901548, -36.07146144527823, -1777.1431630763161]
[0.0, -0.0004920511372371313, -35.403872216337284, -1611.8772045227556]
[0.0, -0.0005217230361602968, -29.12077216635695, -1501.7446713442012]
[0.0, -0.000407485193061001, -25.117616136301773, -1699.86252514217]
[0.0, -0.0006246526979349191, -42.2110923171883, -1474.2046876977493]
[0.0, -0.0004288108089087339, -28.159344315394847, -1512.7288988271905]
[0.0, -0.0004912852792864891, -36.99478305573519, -1582.271320558394]
[0.0, -

[0.0, -0.00044503250993150267, -26.45763107874538, -1585.3336126621004]
[0.0, -0.0005597954122081235, -32.05535955310041, -1767.6647416455055]
[0.0, -0.0005065990784668442, -35.459599407157995, -1398.2490868538991]
[0.0, -0.00046869645967975547, -29.08500056796853, -1833.7204186623965]
[0.0, -0.0005576062005295507, -27.0634065782879, -1246.2911044067853]
[0.0, -0.0005902197687756695, -47.23368131419255, -2092.379537589888]
[0.0, -0.0005791286017625827, -47.699054167125844, -1933.746992104583]
[0.0, -0.0004772413595727441, -34.27396101421241, -1202.1957358950146]
[0.0, -0.000540140718924755, -28.68164365261641, -1033.305367227303]
[0.0, -0.0004345521906182109, -27.85692012491412, -1556.4597745068947]
[0.0, -0.00045928547173566174, -25.821426766868285, -1876.509486880858]
[0.0, -0.00055719495075807, -42.78040712829984, -1719.9855106362147]
[0.0, -0.00040761710286219444, -26.73549785791524, -1397.5643036050074]
[0.0, -0.0006318699183174808, -42.07316678014175, -1766.638961401561]
[0.0, -0

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
: